## OpenAI Function Calling in Nutshell
#### Exploring Function Calling with Azure Open AI. 

In [2]:
# Install the dependencies for this notebook
%pip install -q openai python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [70]:
from openai import AzureOpenAI
import json
import os
from dotenv import load_dotenv
_ = load_dotenv()

Note: Create `.env` file in your root directory and add your Azure Open AI key and Azure Open AI endpoint in it. 


In [71]:
# Configure Azure Open AI
os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
# Details about supported API versions: https://learn.microsoft.com/en-us/azure/ai-services/openai/reference
os.environ["OPENAI_API_VERSION"] = "2024-02-01"
os.environ["OPENAI_API_TYPE"] = "azure"

In [72]:
# initialize the model
llm = AzureOpenAI()

#### `tools` model Parameter

We define a function `"NER"` with the `"tools"` model parameter.

The function returns the Named Entities in the input text. 

Note that the function in this example is a hypothetical one, it's not implemented in code.

In [74]:
#I use the latest gpt-4-turbo-2024-09-04 model

response = llm.chat.completions.create(
    model="gpt-4-2024-09-04",
    temperature=0,
    messages=[
        {"role":"system","content":"Think carefully, and then analyze the text as instructed. If a text doesn't contain any of named entities, just provide an empty object."},
        {"role": "user", "content": "My name is Vlad and I'm working at Microsoft as a Solutions Architect."},
    ],
    tools=[

        {"type": "function",
         "function": {
             'name': 'NER',
             'description': 'Extract all named entities in the provided context. NER stands for Named Entity Recognition',
             'parameters': {'type': 'object',
                            'properties': {'person': {'description': 'person name', 'type': 'string'},
                                           'company': {'description': 'company name', 'type': 'string'},
                                           'job_title': {'description': 'person job title', 'type': 'string'}},
                            'required': ['person', 'company', 'job_title']}}
         }

    ],

)

Let's look at response 

In [75]:
response

ChatCompletion(id='chatcmpl-9OkJbZ0e8C2u93EQRbvZdjxIFryVj', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_pcgXnVErWFZqFULBSEStxpA2', function=Function(arguments='{"person":"Vlad","company":"Microsoft","job_title":"Solutions Architect"}', name='NER'), type='function')]), content_filter_results={})], created=1715684935, model='gpt-4-turbo-2024-04-09', object='chat.completion', system_fingerprint='fp_f5ce1e47b6', usage=CompletionUsage(completion_tokens=26, prompt_tokens=121, total_tokens=147), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])

In [76]:
# The content is empty.
response.choices[0].message.content

In [77]:

response.choices[0].finish_reason

'tool_calls'

In [78]:
# function name
response.choices[0].message.tool_calls[0].function.name

'NER'

In [79]:
# functions arguments
response.choices[0].message.tool_calls[0].function.arguments

'{"person":"Vlad","company":"Microsoft","job_title":"Solutions Architect"}'

First `finish_reason`='tools_calls' means that the model decided to call the function.

The `tool_calls`array contains ChatCompletionMessageToolCall object which in turn has Function object which contains the function name and the parameters passed to the function.

In other words the model not only decided to call the function NER but also extracted the Named Entities from the input context as an input parameters.

The important point here is that the output is well structured and easy to parse or convert to Python dictionary as shown below:
 


In [80]:
# Convert model output to a dictionary
str_out = response.choices[0].message.tool_calls[0].function.arguments
d = json.loads(str_out)
d["company"]

'Microsoft'

What if the text doesn't contain any Named Entities?

In [82]:
response = llm.chat.completions.create(
    model="gpt-4-2024-09-04",
    temperature=0,
    messages=[
        {"role":"system","content":"Think carefully, and then analyze the text as instructed.If a text doesn't contain any of named entities, just provide an empty object."},
        {"role": "user", "content": "Hello how are you today?"},
    ],
    tools=[

        {"type": "function",
           "function": {
             'name': 'NER',
             'description': 'Extract all named entities in the provided context. NER stands for Named Entity Recognition',
             'parameters': {'type': 'object',
                            'properties': {'person': {'description': 'person name', 'type': 'string'},
                                           'company': {'description': 'company name', 'type': 'string'},
                                           'job_title': {'description': 'person job title', 'type': 'string'}},
                            'required': ['person', 'company', 'job_title']}}
         },
    ],

)

In [43]:
response

ChatCompletion(id='chatcmpl-9OjlhAI46FEKSZ5V6zDfnTB9xyztL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to assist you. How can I help you today?", role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1715682833, model='gpt-4-turbo-2024-04-09', object='chat.completion', system_fingerprint='fp_f5ce1e47b6', usage=CompletionUsage(completion_tokens=34, prompt_tokens=84, total_tokens=118), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'v

In [83]:
response.choices[0].message.content

"I'm here to help you! How can I assist you today?"

In [84]:
# Now the finish_reason is : "stop"
response.choices[0].finish_reason

'stop'

In [85]:
# tool_calls is empty
response.choices[0].message.tool_calls

The `finish_reason` is `stop` in this case. This state means the model has a final response and there is no need to call any additonal function. 
We'll use the finish reason to build agentic flows where there are multiple looping calls to the functions and finish_reason serves as a stopping criteria.

Pay attention to the `tool_calls` array which is empty in this case. 

#### Parallel Function calling

What if we have more than one function ? 

Open AI supports `Parallel function calling`, when in one request to the model, the model decides that multiple functions should be called.

The model decides to call the function based on the context and the input text.

In [86]:
#Note that latest gpt-4-turbo-2024-09-04 model doesn't support yet Parallel Function calls, so I use here another gpt-4-turbo version
response = llm.chat.completions.create(
    model="gpt-4-turbo",
    temperature=0,
    messages=[
        {"role":"system","content":"Think carefully, and then analyze the text as instructed"},
        {"role": "user", "content": "My name is Vlad and I'm working at Microsoft as a Solutions Architect. I love OpenAI."},
    ],
    tools=[

        {"type": "function",
           "function": {
             'name': 'NER',
             'description': 'Extract all named entities in the provided context. NER stands for Named Entity Recognition',
             'parameters': {'type': 'object',
                            'properties': {'person': {'description': 'person name', 'type': 'string'},
                                           'company': {'description': 'company name', 'type': 'string'},
                                           'job_title': {'description': 'person job title', 'type': 'string'}},
                            'required': ['person', 'company', 'job_title']}}
         },
        {"type": "function",
           "function": {
                "name": "SentimentAnalysis",
                "description": "Extract sentiment in the provided context. The sentiment value must be on those: `positive`, `negative` or `neutral`.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "sentiment": {"description": "sentiment value","type": "string"
                        }
                    },
                    "required": ["sentiment"]}}
        },
    ],
)

In [87]:
response

ChatCompletion(id='chatcmpl-9OkNHplXc3K1Q3tygBpjTqRADRDCg', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_bDvRTFK5OoCk72yYgdDxFFdO', function=Function(arguments='{"person": "Vlad", "company": "Microsoft", "job_title": "Solutions Architect"}', name='NER'), type='function'), ChatCompletionMessageToolCall(id='call_aftIDnfZWqP6cqOC5JMQALCf', function=Function(arguments='{"sentiment": "positive"}', name='SentimentAnalysis'), type='function')]), content_filter_results={})], created=1715685163, model='gpt-4', object='chat.completion', system_fingerprint='fp_2f57f81c11', usage=CompletionUsage(completion_tokens=98, prompt_tokens=156, total_tokens=254), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered

In [88]:
response.choices[0].finish_reason

'tool_calls'

tool_calls array now contains multiple ChatCompletionMessageToolCall objects, each of which contains a Function object with the function name and the parameters passed to the function.

Each tool has an unique `ID`

In [89]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_bDvRTFK5OoCk72yYgdDxFFdO', function=Function(arguments='{"person": "Vlad", "company": "Microsoft", "job_title": "Solutions Architect"}', name='NER'), type='function'),
 ChatCompletionMessageToolCall(id='call_aftIDnfZWqP6cqOC5JMQALCf', function=Function(arguments='{"sentiment": "positive"}', name='SentimentAnalysis'), type='function')]

In [92]:
for func in response.choices[0].message.tool_calls:
    print(func.function.name)
    print(func.function.arguments)

NER
{"person": "Vlad", "company": "Microsoft", "job_title": "Solutions Architect"}
SentimentAnalysis
{"sentiment": "positive"}


In [93]:
str_out = response.choices[0].message.tool_calls[1].function.arguments
d = json.loads(str_out)
d["sentiment"]

'positive'